In [ ]:
# Step 1: Install necessary libraries
!pip install transformers datasets scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.2 MB/s eta 0:00:00


In [3]:
# mount google drive

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [4]:
# Step 2: Import required modules
import datasets
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Step 3: load data and sample 10K examples
# Replace data file with your own file path.
data = pd.read_csv("/drive/My Drive/Colab Notebooks/train.tsv", sep="\t")
data = data.sample(n=5000, random_state=42)
data = pd.DataFrame(data)
data.rename(columns={"Sentiment": "label", "Phrase": "text"}, inplace=True)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 95722 to 37774
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    5000 non-null   int64 
 1   SentenceId  5000 non-null   int64 
 2   text        5000 non-null   object
 3   label       5000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 195.3+ KB
None


In [5]:
# Step 4: Split the data into train and test subsets (50/50 split)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42
)


In [10]:
# build a LinearSVC model as a baseline comparison to the BERT model
# since LinearSVC is a linear model, we can print out its top features in each category to see whether the model learned something meaningful

from sklearn.feature_extraction.text import CountVectorizer
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')
X_train_svm_vecs = unigram_count_vectorizer.fit_transform(train_texts)

from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1, max_iter=2000)
svm_clf.fit(X_train_svm_vecs, train_labels)
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names_out()))
very_negative_10 = feature_ranks[-10:]
print("Very negative words")
for i in range(0, len(very_negative_10)):
    print(very_negative_10[i])
print()

Very negative words
(1.143363109782535, 'prurient')
(1.1445783810259098, 'whatsoever')
(1.1661680516283428, 'dull')
(1.1745522724403046, 'boring')
(1.2140922668637615, 'plain')
(1.2520157720473044, 'misery')
(1.2834920048903478, 'wastes')
(1.3449220714375143, 'unfocused')
(1.3554180036378427, 'worst')
(1.4527436082284486, 'ugly')



In [13]:
# LinearSVC test performance
# with 4000 training examples, LinearSVC achieved 0.533 accuracy and 0.346 macro-f1.

from sklearn.metrics import f1_score

X_test_svm_vecs = unigram_count_vectorizer.transform(test_texts)
y_pred_svm = svm_clf.predict(X_test_svm_vecs)

# accuracy
svm_acc = svm_clf.score(X_test_svm_vecs,test_labels)
print(svm_acc)

# f1_score
svm_f1 = f1_score(test_labels, y_pred_svm, average = None)
svm_f1_avg = f1_score(test_labels, y_pred_svm, average = 'macro')
print(svm_f1)
print(svm_f1_avg)

0.533
[0.14925373 0.2283737  0.70058382 0.34604106 0.30769231]
0.3463889234052174


In [24]:
from sklearn.metrics import classification_report

y_pred_svm = svm_clf.predict(X_test_svm_vecs)
print(classification_report(test_labels, y_pred_svm))

              precision    recall  f1-score   support

           0       0.23      0.11      0.15        45
           1       0.34      0.17      0.23       191
           2       0.62      0.81      0.70       518
           3       0.37      0.32      0.35       182
           4       0.40      0.25      0.31        64

    accuracy                           0.53      1000
   macro avg       0.39      0.33      0.35      1000
weighted avg       0.49      0.53      0.50      1000



In [14]:
# use the same training data set to fine-tune a BERT model

# Convert the data into Hugging Face Dataset format
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})

# Combine into a DatasetDict for easier handling
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})


In [15]:
unique_labels_count = data['label'].value_counts()
print(unique_labels_count)
print(unique_labels_count.count())

label
2    2522
3    1075
1     859
4     320
0     224
Name: count, dtype: int64
5


In [16]:
num_unique_labels = unique_labels_count.count()
print("Number of unique labels:", num_unique_labels)

Number of unique labels: 5


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:

# Step 5: Load the tokenizer and model

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_unique_labels)  # classification

# Step 6: Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Remove unnecessary columns
tokenized_datasets.set_format("torch")  # Convert to PyTorch tensors






/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
# Step 7: Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred # logits are the raw prediction values. For this task each test example will receive five raw prediction values, one for each category
    predictions = logits.argmax(axis=-1) # the category with highest prediction value/confidence is the final prediction result
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Step 8: Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="distilbert-finetune-kaggle",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Step 9: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# Step 10: Train the model
trainer.train()

# run time about 6min

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.830500,0.927415,0.637000,0.463209,0.421251,0.422540
2,0.715200,0.886938,0.639000,0.555079,0.471134,0.466438


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=500, training_loss=0.856481014251709, metrics={'train_runtime': 367.8857, 'train_samples_per_second': 21.746, 'train_steps_per_second': 1.359, 'total_flos': 1059795886080000.0, 'train_loss': 0.856481014251709, 'epoch': 2.0})

In [20]:
# Step 11: Evaluate the model on the test set
results = trainer.evaluate()
#print("Evaluation Results:", results)

In [22]:
print("macro-f1", results["eval_f1"])
print("acc", results["eval_accuracy"])

macro-f1 0.46643809882312903
acc 0.639


In [40]:
import numpy as np
predictions = trainer.predict(tokenized_datasets["test"])
preds = predictions.predictions  # The raw predictions
y_pred_bert = np.argmax(preds, axis=1)
y_true = predictions.label_ids


In [53]:
# prompt: generate classification report from y_pred and y_true

from sklearn.metrics import classification_report

print("------BERT classification report")
print(classification_report(y_true, y_pred_bert))

print("-------SVM classification report")
print(classification_report(y_true, y_pred_svm))


------BERT classification report
              precision    recall  f1-score   support

           0       0.50      0.02      0.04        45
           1       0.50      0.57      0.53       191
           2       0.77      0.76      0.76       518
           3       0.50      0.62      0.55       182
           4       0.50      0.39      0.44        64

    accuracy                           0.64      1000
   macro avg       0.56      0.47      0.47      1000
weighted avg       0.64      0.64      0.63      1000

-------SVM classification report
              precision    recall  f1-score   support

           0       0.23      0.11      0.15        45
           1       0.34      0.17      0.23       191
           2       0.62      0.81      0.70       518
           3       0.37      0.32      0.35       182
           4       0.40      0.25      0.31        64

    accuracy                           0.53      1000
   macro avg       0.39      0.33      0.35      1000
weighted av

In [50]:
from sklearn.metrics import confusion_matrix

cm_bert = confusion_matrix(y_true, y_pred_bert)
print("BERT Confusion Matrix:\n", cm_bert)

cm_svm = confusion_matrix(y_true, y_pred_svm)
print("SVM Confusion Matrix:\n", cm_svm)

BERT Confusion Matrix:
 [[  1  37   7   0   0]
 [  1 109  65  16   0]
 [  0  64 392  58   4]
 [  0   8  41 112  21]
 [  0   1   2  36  25]]
SVM Confusion Matrix:
 [[  5  12  21   5   2]
 [  9  33 124  20   5]
 [  5  36 420  51   6]
 [  2  13  97  59  11]
 [  1   4  19  24  16]]


In [49]:
# prompt: print out bert prediction errors in each category in the format of test_label, predicted label and the original text

import numpy as np

texts = test_texts.values.tolist()
#print(texts)

for true_label, predicted_label, text in zip(y_true, y_pred_bert, texts):
  if true_label==1 and predicted_label==3:
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}, Text: {text}")


True Label: 1, Predicted Label: 3, Text: The bottom line with Nemesis is the same as it has been with all the films in the series : Fans will undoubtedly enjoy it , and the uncommitted need n't waste their time on it
True Label: 1, Predicted Label: 3, Text: There 's something fishy about a seasonal holiday kids ' movie
True Label: 1, Predicted Label: 3, Text: Memorable for a peculiar malaise that renders its tension flaccid
True Label: 1, Predicted Label: 3, Text: A film that will be best appreciated by those willing to endure its extremely languorous rhythms , Waiting for Happiness
True Label: 1, Predicted Label: 3, Text: Is it really an advantage to invest such subtlety and warmth in an animatronic bear when the humans are acting like puppets ?
True Label: 1, Predicted Label: 3, Text: his character awakens to the notion that to be human
True Label: 1, Predicted Label: 3, Text: Hoffman waits too long to turn his movie in an unexpected direction , and even then his tone retains a gente